## Problem 4.1
***
Using the data file found here: https://alien.usm.uni-muenchen.de/SDSS/specPhotoDR12v3 hoyleb extcorr clean1e5.fit ,Construct an input array of model magnitudes and colors. Construct a
target array using the column name SPEC CLASS ID. Note that the object
types are shown in the column SPEC CLASS and correspond to Stars, Galaxies
and Quasars. Remove all of the Quasars.

Split the data set into a training, test, and validation samples of size (60%,
20%, 20%). Train a Random Forest classifier to perform star galaxy separation.
Make probability predictions for the star class, on the validation data and
make a reliability curve diagram.
Use the sklearn calibration classifier trained on the 
1.  the validation data
2.  the training data
to calibrate the output of the trained machine.

## Problem 4.2
***
Make probabilistic predictions on the test data, and calibrate those probabilities
using the trained calibration classifier in both 1.a) and 1.b).
Now make the reliability curve diagrams for the test data, and show and
label the curves from
1) the uncalibrated probabilities,
2) those calibrated in 1.a)
3) those calibrated in 1.b)

## Problem 4.3
***
Compare the F1 (from sklearn.metrics) scores as measured on the test sample
on all of the calibrated and calibrated predictions
Describe your results with at least 100 words.
***

# Problem 4.1
***

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
from sklearn.model_selection import train_test_split

%matplotlib inline

In [77]:
dataset =Table.read('specPhotoDR12v3_hoyleb_extcorr_clean1e5.fit')
#removing quasar
dataset_woq=dataset[dataset["SPEC_CLASS"] != "QSO   "]

#######defining target 
target =dataset_woq["SPEC_CLASS"]

########defining feature
# we use dereddend magnitudes 
color = ['DERED_U', 'DERED_G', 'DERED_R','DERED_I','DERED_Z']

#and color combinations created from them.
comb= []
for i in range(len(color)):
    for j in range(i, len(color)):
       comb.append(color[i]+'-' + color[j])

# determining value for each color comb
inputs = np.zeros((len(dataset_woq), len(color+comb)), dtype=np.float64) #initiate zero array of size (number of rows x length(mag+col_comb))
for i, key in enumerate(color):
    inputs[:, i] = np.array(dataset_woq[key])

comb_ind = len(color)
for i in range(len(color)):
    for j in range(i, len(color)):
        inputs[:, comb_ind] = np.array(dataset_woq[color[i]] - dataset_woq[color[j]])
        comb_ind += 1

#feature column
features=color+comb



In [91]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import LabelEncoder
#create panda dataframe 
df_input=pd.DataFrame(inputs, columns=features)
df_input=df_input.drop(["DERED_U-DERED_U","DERED_G-DERED_G", "DERED_R-DERED_R", 
                        "DERED_I-DERED_I", "DERED_Z-DERED_Z"], axis=1)

le=LabelEncoder()
df_target =pd.DataFrame({" Object Type": le.fit_transform(target)})

X_train_cv , X_test, y_train_cv, y_test=train_test_split(df_input, df_target, test_size=0.2)
X_train, X_cv, y_train, y_cv = train_test_split(X_train_cv, y_train_cv, test_size=0.25)

In [92]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier()
forest.fit(X_train, y_train)

/home/khoirul_muzakka/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/khoirul_muzakka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [102]:
forest.predict_proba(X_test)

array([[ 1. ,  0. ],
       [ 0.7,  0.3],
       [ 1. ,  0. ],
       ..., 
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ]])